In [1]:
import pandas as pd
import spacy
from sklearn import model_selection
import numpy as np


from tensorflow import keras
from tensorflow.keras import layers, models

In [21]:
df = pd.read_json(open("../../data/data.json", "r", encoding="utf8"))
df

,title,author,year,text,themes
0,"[45, 4, 139, 908]",0,2002,"[2, 2361, 1096, 5, 2361, 297, 9670, 995, 407, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,[2892],1,2002,"[1, 97, 107, 4468, 3, 148, 294, 32, 1426, 8501...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,[208],2,2002,"[107, 19, 716, 21, 1450, 329, 239, 162, 429, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[1220, 40, 115, 2037, 16, 2377]",3,2002,"[329, 304, 18, 72, 8, 1, 8, 1, 44, 1337, 1, 20...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[263, 1220]",4,2002,"[9, 644, 75, 143, 537, 197, 676, 179, 106, 43,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
17069,"[15, 192, 72, 526, 610, 16, 9, 1584]",1030,1990,"[15, 192, 72, 526, 4433, 15, 39, 64, 2997, 271...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
17070,"[15, 115]",3224,1990,"[115, 18, 72, 125, 610, 2, 801, 1, 98, 1126, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
17071,"[15, 25, 1, 12]",3466,2005,"[15, 1, 12, 36, 35, 15, 1, 315, 176, 537, 38, ...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
17072,"[278, 608]",4046,1990,"[40, 486, 686, 238, 40, 486, 8491, 407, 406, 1...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [3]:
texts_len = df['text'].apply(len)
df.drop(df[texts_len<50].index, inplace=True)

In [4]:
max_features = 10000  # maximum number of words in vocabulari 5000
max_len = 150  # max length of string

In [5]:
joined_text = df['title'] + df['text']
X = keras.preprocessing.sequence.pad_sequences(list(joined_text), maxlen=max_len, padding='post')
Y = np.array(df['year'].to_list())

In [6]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.1, random_state=42)

### Model 1 simple Dense Layers

In [73]:
'''model = keras.Sequential([
    layers.BatchNormalization(input_shape=input_shape),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3), 
    layers.Dense(1, activation='sigmoid'),
])'''

embedding_dim =128
model1 = keras.models.Sequential([
  keras.layers.Embedding(input_dim=max_features,
                           output_dim=embedding_dim,
                           input_length=max_len),
  keras.layers.Flatten(),
  keras.layers.Dense(2000,activation='relu'),
  keras.layers.Dense(500,activation='relu'),
  keras.layers.Dense(100,activation='relu'),
  keras.layers.Dense(1, activation='relu')
])

model1.compile(optimizer='nadam',
              loss='mean_squared_error',
              metrics=['MAE'])

model1.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 150, 128)          1280000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 19200)             0         
_________________________________________________________________
dense_36 (Dense)             (None, 2000)              38402000  
_________________________________________________________________
dense_37 (Dense)             (None, 500)               1000500   
_________________________________________________________________
dense_38 (Dense)             (None, 100)               50100     
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 101       
Total params: 40,732,701
Trainable params: 40,732,701
Non-trainable params: 0
__________________________________________

In [74]:
epochs = 5
model1.fit(np.array(X_train), np.array(Y_train),
          #batch_size=128,
          validation_data=(np.array(X_test),np.array(Y_test)),
          epochs=epochs)

Epoch 1/5
373/373 [==============================] - 163s 437ms/step - loss: 139711.5781 - MAE: 195.3840 - val_loss: 36333.7109 - val_MAE: 172.5712
Epoch 2/5
373/373 [==============================] - 162s 435ms/step - loss: 17208.5430 - MAE: 105.7098 - val_loss: 9187.5400 - val_MAE: 64.4781
Epoch 3/5
373/373 [==============================] - 164s 438ms/step - loss: 11283.1445 - MAE: 83.2191 - val_loss: 13769.0459 - val_MAE: 95.1717
Epoch 4/5
373/373 [==============================] - 162s 434ms/step - loss: 9514.2002 - MAE: 78.6174 - val_loss: 6860.0107 - val_MAE: 52.6344
Epoch 5/5
373/373 [==============================] - 161s 432ms/step - loss: 9637.3887 - MAE: 78.8965 - val_loss: 27867.0664 - val_MAE: 148.1196


In [9]:
score1 = model1.evaluate(np.array(X_test), np.array(Y_test)) 

print("Test Score:", score1[0])
print("Test Accuracy:", score1[1])

42/42 [==============================] - 1s 15ms/step - loss: 12033.6885 - MAE: 88.8316 - accuracy: 0.0000e+00
Test Score: 12033.6884765625
Test Accuracy: 88.83157348632812


In [57]:
X2 = [[row.author] + row.themes for index, row in df.iterrows()]
X2 = np.array(df.author)
Y2 = np.array(df.year)

In [58]:

model2 = keras.models.Sequential([
  keras.layers.Dense(units=2000, activation='relu', input_shape=[1]),
  
  keras.layers.Dense(2000,activation='relu'),
  keras.layers.Dense(500,activation='relu'),
  keras.layers.Dense(100,activation='relu'),
  keras.layers.Dense(1, activation='relu')
])



model2.build()

In [59]:
model2.compile(optimizer='nadam',
              loss='mean_squared_error',
              metrics=['MAE'])

In [60]:
model2.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 2000)              4000      
_________________________________________________________________
dense_32 (Dense)             (None, 2000)              4002000   
_________________________________________________________________
dense_33 (Dense)             (None, 500)               1000500   
_________________________________________________________________
dense_34 (Dense)             (None, 100)               50100     
_________________________________________________________________
dense_35 (Dense)             (None, 1)                 101       
Total params: 5,056,701
Trainable params: 5,056,701
Non-trainable params: 0
_________________________________________________________________


In [61]:
X2_train, X2_test, Y2_train, Y2_test = model_selection.train_test_split(X2, Y2, test_size=0.1, random_state=42)

In [62]:
model2.fit(X2_train, Y2_train, epochs=5, batch_size=128, validation_split=0.1)

Epoch 1/5
107/107 [==============================] - 13s 125ms/step - loss: 3974632.5000 - MAE: 1992.9155 - val_loss: 3975730.2500 - val_MAE: 1993.3551
Epoch 2/5
107/107 [==============================] - 13s 123ms/step - loss: 3974631.0000 - MAE: 1992.9154 - val_loss: 3975730.2500 - val_MAE: 1993.3551
Epoch 3/5
107/107 [==============================] - 14s 127ms/step - loss: 3974631.2500 - MAE: 1992.9155 - val_loss: 3975730.2500 - val_MAE: 1993.3551
Epoch 4/5
107/107 [==============================] - 15s 138ms/step - loss: 3974631.7500 - MAE: 1992.9147 - val_loss: 3975730.2500 - val_MAE: 1993.3551
Epoch 5/5
107/107 [==============================] - 14s 134ms/step - loss: 3974630.5000 - MAE: 1992.9155 - val_loss: 3975730.2500 - val_MAE: 1993.3551


In [63]:
accr = model2.evaluate(X2_test,Y2_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

53/53 [==============================] - 0s 9ms/step - loss: 3982072.5000 - MAE: 1994.9067
Test set
  Loss: 3982072.500
  Accuracy: 1994.907


In [72]:
df[['author', 'year']].corr()

,author,year
author,1.000000,0.170517
year,0.170517,1.000000


In [15]:
'''output_dim = 100

inputs = keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, output_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(len(Y[0]), activation='softmax', name="predictions")(x)

model3 = keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 0,073'''


'output_dim = 100\n\ninputs = keras.Input(shape=(None,), dtype="int64")\n\n# Next, we add a layer to map those vocab indices into a space of dimensionality\n# \'embedding_dim\'.\nx = layers.Embedding(max_features, output_dim)(inputs)\nx = layers.Dropout(0.5)(x)\n\n# Conv1D + global max pooling\nx = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)\nx = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)\nx = layers.GlobalMaxPooling1D()(x)\n\n# We add a vanilla hidden layer:\nx = layers.Dense(128, activation="relu")(x)\nx = layers.Dropout(0.5)(x)\n\npredictions = layers.Dense(len(Y[0]), activation=\'softmax\', name="predictions")(x)\n\nmodel3 = keras.Model(inputs, predictions)\n\n# Compile the model with binary crossentropy loss and an adam optimizer.\nmodel3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 0,073'

In [17]:
output_dim = 100

inputs = keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, output_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(1, activation='relu', name="predictions")(x)

model3 = keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model3.compile(loss="mean_squared_error", optimizer="adam", metrics=["MAE"])


In [18]:
model3.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
dropout (Dropout)            (None, None, 100)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         89728     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         114816    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512 

In [19]:
batch_size = 64
epochs = 5
model3.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Epoch 1/5
168/168 [==============================] - 5s 31ms/step - loss: 879658.0000 - MAE: 662.8161 - val_loss: 26461.2012 - val_MAE: 122.4678
Epoch 2/5
168/168 [==============================] - 5s 31ms/step - loss: 96305.1719 - MAE: 248.0609 - val_loss: 21862.8398 - val_MAE: 119.2258
Epoch 3/5
168/168 [==============================] - 5s 30ms/step - loss: 84816.7266 - MAE: 233.0055 - val_loss: 23970.6250 - val_MAE: 132.5528
Epoch 4/5
168/168 [==============================] - 5s 30ms/step - loss: 84697.7891 - MAE: 232.4709 - val_loss: 25295.7871 - val_MAE: 139.6100
Epoch 5/5
168/168 [==============================] - 5s 31ms/step - loss: 84621.0000 - MAE: 232.1503 - val_loss: 8951.3477 - val_MAE: 65.0715


In [20]:
accr = model3.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

42/42 [==============================] - 0s 4ms/step - loss: 9842.3105 - MAE: 66.0028
Test set
  Loss: 9842.311
  Accuracy: 66.003
